Con respecto al cuaderno de **Transformers**:
- Comparar 3 Transformers *multilingual*
- Comparar 3 por *separado* (español/inglés), es decir 3 de español y 3 de inglés
    - Otra opción es traducir y pasárselo al modelo del otro idioma. 
- El mejor de los dos apartados anteriores le hacemos optimización de hiperparámetros (5 o 6 hiperparámetros)

In [17]:
# Instalacion de librerias
import random
import torch
import numpy as np
import os
from pytorch_lightning import seed_everything
import matplotlib.pyplot as plt
import seaborn as sns
import re

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after eachepoch so we can plot them.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["TF_DETERMINISTIC_OPS"] = "1" # See:https://github.com/NVIDIA/tensorflow-determinism#confirmed-current-gpu-specific-sources-of-non-determinism-with-solutions
seed_everything(42, workers=True)

from datasets import Dataset, DatasetDict #, load_metric EN PRINCIPIO ESTÁ DESCONTINUADO, TENDREMOS QUE BUSCAR OTRA ALTERNATIVA
import pandas as pd
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
TrainingArguments, Trainer, pipeline, EarlyStoppingCallback
from huggingface_hub import login

Seed set to 42


In [2]:
# Comprobacion GPU
# Check that pyTorch is identifying the GPU
if torch.cuda.device_count() > 0:
    # If a GPU is available, print its name
    print(f'GPU detected. Currently using: "{torch.cuda.get_device_name(0)}"')
    # Set the device to GPU for accelerated computations
    device = torch.device("cuda")
else:
    # If no GPU is available, inform the user to change the runtime type
    print('Currently using CPU. To utilize GPU acceleration, change the runtime type in the \'runtime\' tab.')

GPU detected. Currently using: "NVIDIA GeForce RTX 4070"


# Lectura y etiquetado de Datos

In [10]:
# ================================
# 1. Carga y preparación de datos
# ================================
# Cargar dataset de entrenamiento con etiquetas
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})
df = df.drop(columns=["Unnamed: 0"])
df["label"] = df["label"].astype(int)

# from sklearn.model_selection import train_test_split
# train_df, eval_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)

# train_dataset = Dataset.from_pandas(train_df)
# eval_dataset = Dataset.from_pandas(eval_df)

# PENDIENTE DE DEFINIR UN FICHERO TRAIN, VAL Y TEST
# División estratificada
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Distribución fichero de entrenamiento:")
print(train_df.value_counts('label'))

print("Distribución fichero de test:")
print(test_df.value_counts('label'))

print("Distribución fichero de validación:")
print(val_df.value_counts('label'))


Distribución fichero de entrenamiento:
label
0    914
1    841
Name: count, dtype: int64
Distribución fichero de test:
label
0    196
1    181
Name: count, dtype: int64
Distribución fichero de validación:
label
0    196
1    180
Name: count, dtype: int64


# Preprocesado del texto

**¿Es realmente necesario preprocesar una transcripción?**

In [6]:
# Para ver el vocabulario del dataset

import pandas as pd

# Suponiendo que la columna de texto se llama 'text'
# Tokeniza y obtiene todas las palabras
vocabulario = set(
    palabra.lower()
    for fila in df["text"].dropna()
    for palabra in fila.split()
)

# Mostrar todo el vocabulario
print(vocabulario)

# (Opcional) Ver el número de palabras únicas
print(f"\nTotal de palabras únicas: {len(vocabulario)}")


{'distante', 'fácilmente.', 'bandas.', 'reclamar', 'vgien', 'nació,', 'semantics,', 'smithaa,', 'capa.', 'calhild,', '"nice', 'cursitos', 'aprenda', 'direcig', 'habits.', '“i', 'shoot.', 'beach_with_umbrella', 'información."', 'hablo,', 'communities', 'codoa|', 'drivers', 'amenazado.', 'seduce.', 'beware,', 'situation.', '"she\'s', 'deseen', 'prominent', 'caminacostina,', 'chick', 'identificadas', 'parezca.', 'casi', 'police_car_light', 'relationship', 'cuadrilátero.', 'trébete', 'dormida!', 's3x-based', 'hereje', 'fractura', 'bone', 'provoc', 'ucuyor', 'perrita', 'línea', 'llamada', 'nan', 'vavidsok.', 'bridgerton.', 'llorar.', 'patricio,', 'refiere', 'anduviera', 'emprendedoras.', 'desinfectamos', '¡suscríbete!', 'viewing', "'jokzstaa", '""bebe"",', 'hue,', 'noelle', 'katyas2', 'poder,', 'hablé', 'surprised,', 'burn', 'demonizar', 'cabrera?', 'mode,', 'ambition,', 'comprometida.', '1970,', 'hay.', 'obvious,', 'zane,', 'hey', 'contraseña', 'funcionaba,', 'narcisista', 'fueras', 'tying

In [3]:
import re

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)        # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)     # remove bitly links
    tweet = re.sub(r'\[link\]', '', tweet )      # remove [link]
    tweet = re.sub(r'\[url\]', '', tweet )       # remove [url]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove tweeted at
    tweet = re.sub(r'\[user\]', '', tweet )                      # remove [user]
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)      # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def remove_emojis(tweet):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00010000-\U0010FFFF"
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\ufe0f"  # dingbats

                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3208/1710960392.py:14: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet


Convertimos todo a minúsculas

In [12]:
campo_texto = 'text'

train_df[campo_texto] = train_df[campo_texto].str.lower()
val_df[campo_texto] = val_df[campo_texto].str.lower()
test_df[campo_texto] = test_df[campo_texto].str.lower()

# Definición de métricas

In [13]:
# Función para realizar distintas métricas en ejecución

def compute_metrics(eval_pred):

  ##############
  ## predictions son logits, que son tuplas de la forma [valor1, valor2]
  ## Por ejemplo [-1.5606991,  1.6122842] significa que ha predicho eso para un documento
  ## Eso es lo que pasa a la última capa del transformer (softmax si es binario)
  ## Por eso se utiliza el índice del valor máximo de la tupla, para decir que esa es la clase que predice

  ## label_ids = [0, 1, 1, 0, 1]  # Etiquetas reales
  ## predictions = [
  ##  [0.8, 0.2],  # Predicciones para la primera instancia
  ##  [0.3, 0.7],  # Predicciones para la segunda instancia
  ##  [0.1, 0.9],  # Predicciones para la tercera instancia
  ##  [0.9, 0.1],  # Predicciones para la cuarta instancia
  ##  [0.4, 0.6],  # Predicciones para la quinta instancia
  ##           ]

  ##############

  labels = eval_pred.label_ids
  preds = eval_pred.predictions.argmax(-1)

  # Compute precision, recall, F1-score, and support
  precision, recall, f1, _ = sk.metrics.precision_recall_fscore_support(labels, preds, average="macro")

  # Calculate F1-score for the minority class (label = 1)
  f1_minoritaria= f1_score(labels, preds, pos_label=1)

  # Calculate F1-score for the majority class (label = 0)
  f1_mayoritaria = f1_score(labels, preds, pos_label=0)

  # Calculate accuracy
  acc = sk.metrics.accuracy_score(labels, preds)

  # Calculate Area Under the Curve (AUC)
  AUC = roc_auc_score(labels, preds)

  # Calculate Precision-Recall Area Under the Curve (AUC)
  PREC_REC = average_precision_score(labels, preds)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'AUC': AUC,
      'f1_minoritaria': f1_minoritaria,
      'f1_mayoritaria': f1_mayoritaria,
      'PREC_REC': PREC_REC
  }

# Entrenamiento del modelo

In [19]:
# Cargamos el token de HuggingFace que lo tenemos en un fichero oculto e iniciamos sesión

with open("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/huggingfaceToken.txt", "r") as file:
    hf_token = file.read().strip()

login(hf_token)

In [20]:
# Eleccion del modelo
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = 'bert-base-multilingual-uncased'
#model_checkpoint = 'annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [21]:
# Se carga el modelo preentrenado
n_labels = 2

# El uso de una función de inicialización facilita la repetición del entrenamiento
# Se puede usar la misma función de inicialización en diferentes ejecuciones del código o en configuraciones de entrenamiento diferentes
# Esto facilita la repetición del entrenamiento y la reproducibilidad, ya que se puede inicializar el modelo
# de la misma manera en cada ejecución.

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                              num_labels = n_labels) #, return_dict = True )
                                                              # use_auth_token = 'token propio de HugginFace')

In [22]:
# Para saber el nombre del modelo
model_name = model_checkpoint.split("/")[-1]
model_name

'bert-base-multilingual-uncased'

## Fine-tuning

In [23]:
# Selección de hiperparámetros
BATCH_SIZE = 32
NUM_TRAIN_EPOCHS = 15
LEARNING_RATE = 3e-5
MAX_LENGTH = 128
WEIGHT_DECAY = 0.1

In [24]:
def tokenize_data(examples):
  return tokenizer(examples[campo_texto], truncation=True, max_length=MAX_LENGTH, padding=True)

In [31]:
from transformers import Trainer, TrainingArguments

optim = ["adamw_hf", "adamw_torch", "adamw_apex_fused", "adafactor", "adamw_torch_xla"]
MAX_LENGTH = 128

training_args = TrainingArguments(
    output_dir = 'results',
    num_train_epochs = NUM_TRAIN_EPOCHS,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    load_best_model_at_end = True,
    metric_for_best_model = 'recall', # Cambiar la metrica por la que queremos ajustar
    weight_decay = WEIGHT_DECAY,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit = 3,
    optim = optim[1],
    push_to_hub = False
)

output_dir = '/home/adrian/Escritorio/DeepSexist/TrainingBooks/Task3.1/Research/ModelosTransformers'

# Entrenamiento de cada modelo por separado
model_output_dir = f"{output_dir}/modelo_{model_checkpoint}"

# Define el conjunto de datos de entrenamiento específico para esta tarea
train_dataset = train_df['label']
valid_dataset = val_df

# train_dataset = Dataset.from_pandas(train_df)
# valid_dataset = Dataset.from_pandas(val_df)

train_dataset = Dataset.from_pandas(train_dataset)

# Reseteamos el formato para que no haya fallos
train_dataset.reset_format()
valid_dataset.reset_format()

columns_train = train_dataset.column_names  # Coge todas las columnas
columns_valid = valid_dataset.column_names  # Coge todas las columnas
columns_train.remove("labels") # Elimina la columna "labels"
columns_valid.remove("labels") # Elimina la columna "labels"

# Hace la tokenización y elimina todas las columnas que no se necesitan
encoded_train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=columns_train)
encoded_valid_dataset = valid_dataset.map(tokenize_data, batched=True, remove_columns=columns_valid)

# Inicializa el entrenador
trainer = Trainer(
    model_init = model_init,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    tokenizer=tokenizer,
)

# Entrena el modelo
trainer.train()

# Guarda el modelo entrenado
trainer.save_model(model_output_dir)

/home/adrian/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
# Convertimos a Dataset de HuggingFace
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(val_df)

# Reseteamos el formato por si viene con formato anterior
train_dataset.reset_format()
valid_dataset.reset_format()

# Eliminamos columnas innecesarias excepto "labels" y texto (ej. "text")
columns_train = train_dataset.column_names
columns_valid = valid_dataset.column_names
columns_train.remove("label")
columns_valid.remove("label")

# Tokenizamos los datasets
encoded_train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=columns_train)
encoded_valid_dataset = valid_dataset.map(tokenize_data, batched=True, remove_columns=columns_valid)

# Hiperparámetros
training_args = TrainingArguments(
    output_dir='results',
    num_train_epochs=NUM_TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    load_best_model_at_end=True,
    metric_for_best_model='recall',  # Puedes usar 'accuracy' o 'f1'
    weight_decay=WEIGHT_DECAY,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    optim="adamw_torch",  # Puedes cambiarlo por otro optimizador si prefieres
    push_to_hub=False
)

# Entrenador
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Entrenamiento
trainer.train()

# Guardar el modelo
output_dir = '/home/adrian/Escritorio/DeepSexist/TrainingBooks/Task3.1/Research/ModelosTransformers' + model_checkpoint
trainer.save_model(output_dir)

Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

Map:   0%|          | 0/376 [00:00<?, ? examples/s]

/home/adrian/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3208/4199679674.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

{'eval_loss': 0.8155430555343628, 'eval_accuracy': 0.6728723404255319, 'eval_f1': 0.6728144830952735, 'eval_precision': 0.6751812366737739, 'eval_recall': 0.6746598639455783, 'eval_AUC': 0.6746598639455783, 'eval_f1_minoritaria': 0.6771653543307087, 'eval_f1_mayoritaria': 0.6684636118598383, 'eval_PREC_REC': 0.5955885466285593, 'eval_runtime': 0.7293, 'eval_samples_per_second': 515.573, 'eval_steps_per_second': 16.454, 'epoch': 4.0}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': 1.032497525215149, 'eval_accuracy': 0.6702127659574468, 'eval_f1': 0.6699793312381437, 'eval_precision': 0.6700056593095642, 'eval_recall': 0.670294784580499, 'eval_AUC': 0.6702947845804988, 'eval_f1_minoritaria': 0.6612021857923497, 'eval_f1_mayoritaria': 0.6787564766839378, 'eval_PREC_REC': 0.5942207478583594, 'eval_runtime': 0.7266, 'eval_samples_per_second': 517.495, 'eval_steps_per_second': 16.516, 'epoch': 5.0}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': 1.358781337738037, 'eval_accuracy': 0.6835106382978723, 'eval_f1': 0.6835083996463307, 'eval_precision': 0.6845587193653492, 'eval_recall': 0.6846371882086169, 'eval_AUC': 0.6846371882086167, 'eval_f1_minoritaria': 0.6826666666666666, 'eval_f1_mayoritaria': 0.6843501326259946, 'eval_PREC_REC': 0.6050784991210524, 'eval_runtime': 0.7284, 'eval_samples_per_second': 516.194, 'eval_steps_per_second': 16.474, 'epoch': 6.0}
{'train_runtime': 83.1999, 'train_samples_per_second': 316.407, 'train_steps_per_second': 9.916, 'train_loss': 0.41995789498993846, 'epoch': 6.0}
